In [1]:
import tensorflow as tf
import numpy as np
import argparse
import IPython.display as display
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
import pandas as pd
tf.compat.v1.enable_eager_execution()
import cProfile
from sklearn.model_selection import train_test_split
import copy as cp
from sklearn.model_selection import KFold
import argparse
import torch
import numpy
tf.keras.backend.set_floatx('float64')

In [2]:
## imports IRM_games_regression to fit C-LRG
from irm_games_regression import * 

In [3]:
path_name = "../ERM_per_env/IRMv1_regression" #(path name to inside IRMv1_regression folder )
import sys
sys.path.insert(1, path_name)
from sem_Sep8 import ChainEquationModel
from models_crossval_Sep8 import *
# from IRM_exhaustive_argparse_hper_wo_disc_Sep8 import *
# from main_v1 import *

In [4]:
from IRM_exhaustive_argparse_hper_wo_disc_Sep8 import *


In [5]:
class erm_model_per_env:
    def __init__(self, model_list,  num_epochs, batch_size, learning_rate):
        
        self.model_list        = model_list
        self.num_epochs   = num_epochs
        self.batch_size   = batch_size
        self.learning_rate=learning_rate
        self.model_final_list = []
        self.index_diff_list = []
        self.w_final_list         = []
        self.w_final_select       = []
         
    
    def fit(self, data_tuple_list, diff_threshold):
        learning_rate = self.learning_rate
        num_epochs = self.num_epochs
        n_e  = len(data_tuple_list)
        
        model_list = self.model_list
    ### fit the model
        for i in range(n_e):
            print (i)
            x_in = data_tuple_list[i][0]
            y_in = data_tuple_list[i][1]
            batch_size = self.batch_size


            model_list[i].compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss="mean_squared_error", metrics=["mean_squared_error"
                                                         ])

            model_list[i].fit(x_in, y_in,  epochs=num_epochs, batch_size=batch_size)
  

        self.model_list = model_list
    
#         diff_threshold = self.diff_threshold
        
        n_f = np.shape(model_list[0].weights[0].numpy())[0]
        index_sel = []
        vec_diff = np.abs(erm_per_env.model_list[1].weights[0].numpy().T-erm_per_env.model_list[0].weights[0].numpy().T)[0]
        index_diff = np.where(vec_diff<diff_threshold)[0]
        print (index_diff)
        x_in_c = data_tuple_list[0][0]
        for i in range(1,n_e):
            x_c = data_tuple_list[i][0]
            x_in_c = np.concatenate((x_in_c, x_c), axis=0)
        y_in_c = data_tuple_list[0][1]
        for i in range(1,n_e):
            y_c = data_tuple_list[i][1]
            y_in_c = np.concatenate((y_in_c, y_c), axis=0)
        e_in_c = data_tuple_list[0][2]
        for i in range(1,n_e):
            e_c = data_tuple_list[i][2]
            e_in_c = np.concatenate((e_in_c, e_c), axis=0) 
        
        x_in_c1 = x_in_c[:,index_diff]
        model_final = keras.Sequential([
            keras.layers.Dense(1)])
        
        model_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
 loss="mean_squared_error", metrics=["mean_squared_error"])

        model_final.fit(x_in_c1, y_in_c,  epochs=num_epochs, batch_size=batch_size)
        
        self.model_final_list.append(model_final)
        self.index_diff_list.append(index_diff) 
        m = np.shape(x_in_c)[1]
        print ("m: " + str(m))
        w_final = np.zeros(m)
        
        w_final[index_diff] = model_final.weights[0].numpy().T[0]
        self.w_final_list.append(w_final)
    
    def validate_fit(self, data_tuple_list, data_tuple_list_valid, threshold_list):
        
        n_e  = len(data_tuple_list)
        x_in_c = data_tuple_list_valid[0][0]
        for i in range(1,n_e):
            x_c = data_tuple_list_valid[i][0]
            x_in_c = np.concatenate((x_in_c, x_c), axis=0)
        y_in_c = data_tuple_list_valid[0][1]
        for i in range(1,n_e):
            y_c = data_tuple_list_valid[i][1]
            y_in_c = np.concatenate((y_in_c, y_c), axis=0)
        e_in_c = data_tuple_list_valid[0][2]
        for i in range(1,n_e):
            e_c = data_tuple_list_valid[i][2]
            e_in_c = np.concatenate((e_in_c, e_c), axis=0) 
        
#         x_in_c1 = x_in_c[:,index_diff]
        
        ind = 0
        cur_error_min = 1e7
        for threshold in threshold_list:
            loss_object = tf.keras.losses.MeanSquaredError()
            self.fit(data_tuple_list, threshold)
            index_diff = self.index_diff_list[ind]
            model_cur = self.model_final_list[ind]
            error = loss_object(y_in_c, model_cur(x_in_c[:,index_diff]))
            if(error<=cur_error_min):
                error = cur_error_min
                ind_min = ind
        
        self.w_final_select.append(self.w_final_list[ind_min])
            
        
        
    def evaluate(self, data_tuple_test):
        ##### evaluations jmtd
        x_test = data_tuple_test[0]
        y_test = data_tuple_test[1]
        x_in   = self.x_in
        y_in   = self.y_in
        
        model = self.model
        train_accuracy= tf.keras.metrics.SparseCategoricalAccuracy()
        test_accuracy= tf.keras.metrics.SparseCategoricalAccuracy()

        ytr_ = model.predict(x_in)
        train_acc =  np.float(train_accuracy(y_in, ytr_))

        yts_ = model.predict(x_test)

        test_acc  =  np.float(test_accuracy(y_test, yts_))
        
        self.train_acc = train_acc
        self.test_acc  = test_acc

In [ ]:
## Run ERM
n_samples = 1000
args = {"seed":8,"setup_sem": "chain", "dim": 10, "n_samples": n_samples, "n_reps": 1, "skip_reps": 0, "print_vectors": 1, "n_iterations": 100000, "lr": 1e-3, "verbose": 0,  "alpha": 0.05, "setup_scramble":0, "setup_hetero":1, "setup_hidden":1, "env_list":[0.2,2.0], "verbose":0, "child":0, "noise_identity":1, "ones":1}

args["methods"] = "ERM"
all_solutions, all_environments, msolution, sem_solution = run_experiment_ERM(args)
true_sol = sem_solution.detach().numpy().T[0]
ERM_sol = msolution.detach().numpy()[0]
print ("Distance of ERM soln " +str(np.linalg.norm(ERM_sol- true_sol)))

In [7]:

## Run C-LRG 
data_tuple_list = convert_regn_np_format(all_environments)
n_e=2
model_list = [] 
for e in range(n_e):

    model_list.append(keras.Sequential([
            keras.layers.Dense(1)
    ]))

num_epochs       = 500
batch_size       = 128
learning_rate    = 0.005
bound            = 2
plot_flag        = 'false'
F_game = fixed_irm_game_model_regression(model_list, learning_rate, num_epochs, batch_size, bound, plot_flag) 


## Run C-LRG
F_game.fit(data_tuple_list)
fgamesol = 0.0
for e in range(n_e):
    fgamesol += F_game.model_list[e].weights[0]
print ("Distance of C-LRG soln " + str(np.linalg.norm(fgamesol.numpy().T[0]- true_sol)))

Distance of C-LRG soln 0.22399556259984105


In [8]:
## Run ERM-per-environment
n_e=2
n_samples = 800
args = {"seed":1,"setup_sem": "chain", "dim": 10, "n_samples": n_samples, "n_reps": 1, "skip_reps": 0, "print_vectors": 1, "n_iterations": 100000, "lr": 1e-3, "verbose": 0,  "alpha": 0.05, "setup_scramble":0, "setup_hetero":1, "setup_hidden":1, "env_list":[0.2,2.0], "verbose":0, "child":0, "noise_identity":0, "ones":1}

args["methods"] = "ERM"
all_solutions, all_environments, msolution, sem_solution = run_experiment_ERM(args)
true_sol = sem_solution.detach().numpy().T[0]
ERM_sol = msolution.detach().numpy()[0]
print (np.linalg.norm(ERM_sol- true_sol))


data_tuple_list = convert_regn_np_format(all_environments)

n_samples = 200
args = {"seed":1,"setup_sem": "chain", "dim": 10, "n_samples": n_samples, "n_reps": 1, "skip_reps": 0, "print_vectors": 1, "n_iterations": 100000, "lr": 1e-3, "verbose": 0,  "alpha": 0.05, "setup_scramble":0, "setup_hetero":1, "setup_hidden":1, "env_list":[0.2,2.0], "verbose":0, "child":0, "noise_identity":0, "ones":1}

args["methods"] = "ERM"
all_solutions, all_environments, msolution, sem_solution = run_experiment_ERM(args)
true_sol = sem_solution.detach().numpy().T[0]
ERM_sol = msolution.detach().numpy()[0]
print (np.linalg.norm(ERM_sol- true_sol))

data_tuple_list_valid= convert_regn_np_format(all_environments)


model_list = [] 
for e in range(n_e):

    model_list.append(keras.Sequential([
            keras.layers.Dense(1)
    ]))
num_epochs       = 200
batch_size       = 128
learning_rate    = 0.005
bound            = 5
plot_flag        = 'false'
erm_per_env      = erm_model_per_env(model_list,  num_epochs, batch_size, learning_rate) 
erm_per_env.validate_fit(data_tuple_list, data_tuple_list_valid, [0.05,0.1,0.2])
# erm_per_env.fit(data_tuple_list, 0.1)

print ("Distance of ERM per env soln "+ str(np.linalg.norm(erm_per_env.w_final_select[0]-true_sol)))

ones1
noise_identity 0
hetero 1
hetero 1
0.19464684
ones1
noise_identity 0
hetero 1
hetero 1
0.51196295
0
Epoch 1/200
7/7 [==============================] - 0s 738us/step - loss: 1.0861 - mean_squared_error: 1.0861
Epoch 2/200
7/7 [==============================] - 0s 658us/step - loss: 0.9707 - mean_squared_error: 0.9707
Epoch 3/200
7/7 [==============================] - 0s 646us/step - loss: 0.8652 - mean_squared_error: 0.8652
Epoch 4/200
7/7 [==============================] - 0s 634us/step - loss: 0.7746 - mean_squared_error: 0.7746
Epoch 5/200
7/7 [==============================] - 0s 771us/step - loss: 0.6926 - mean_squared_error: 0.6926
Epoch 6/200
7/7 [==============================] - 0s 743us/step - loss: 0.6228 - mean_squared_error: 0.6228
Epoch 7/200
7/7 [==============================] - 0s 834us/step - loss: 0.5672 - mean_squared_error: 0.5672
Epoch 8/200
7/7 [==============================] - 0s 684us/step - loss: 0.5154 - mean_squared_error: 0.5154
Epoch 9/200
7/7 [=====

7/7 [==============================] - 0s 689us/step - loss: 0.1894 - mean_squared_error: 0.1894
Epoch 75/200
7/7 [==============================] - 0s 626us/step - loss: 0.1894 - mean_squared_error: 0.1894
Epoch 76/200
7/7 [==============================] - 0s 671us/step - loss: 0.1894 - mean_squared_error: 0.1894
Epoch 77/200
7/7 [==============================] - 0s 729us/step - loss: 0.1894 - mean_squared_error: 0.1894
Epoch 78/200
7/7 [==============================] - 0s 669us/step - loss: 0.1896 - mean_squared_error: 0.1896
Epoch 79/200
7/7 [==============================] - 0s 687us/step - loss: 0.1893 - mean_squared_error: 0.1893
Epoch 80/200
7/7 [==============================] - 0s 707us/step - loss: 0.1892 - mean_squared_error: 0.1892
Epoch 81/200
7/7 [==============================] - 0s 660us/step - loss: 0.1891 - mean_squared_error: 0.1891
Epoch 82/200
7/7 [==============================] - 0s 605us/step - loss: 0.1890 - mean_squared_error: 0.1890
Epoch 83/200
7/7 [=====

7/7 [==============================] - 0s 707us/step - loss: 0.1890 - mean_squared_error: 0.1890
Epoch 149/200
7/7 [==============================] - 0s 709us/step - loss: 0.1890 - mean_squared_error: 0.1890
Epoch 150/200
7/7 [==============================] - 0s 717us/step - loss: 0.1889 - mean_squared_error: 0.1889
Epoch 151/200
7/7 [==============================] - 0s 674us/step - loss: 0.1889 - mean_squared_error: 0.1889
Epoch 152/200
7/7 [==============================] - 0s 593us/step - loss: 0.1888 - mean_squared_error: 0.1888
Epoch 153/200
7/7 [==============================] - 0s 658us/step - loss: 0.1888 - mean_squared_error: 0.1888
Epoch 154/200
7/7 [==============================] - 0s 691us/step - loss: 0.1889 - mean_squared_error: 0.1889
Epoch 155/200
7/7 [==============================] - 0s 581us/step - loss: 0.1890 - mean_squared_error: 0.1890
Epoch 156/200
7/7 [==============================] - 0s 668us/step - loss: 0.1889 - mean_squared_error: 0.1889
Epoch 157/200
7

7/7 [==============================] - 0s 793us/step - loss: 25.9728 - mean_squared_error: 25.9728
Epoch 22/200
7/7 [==============================] - 0s 674us/step - loss: 25.5899 - mean_squared_error: 25.5899
Epoch 23/200
7/7 [==============================] - 0s 641us/step - loss: 25.2198 - mean_squared_error: 25.2198
Epoch 24/200
7/7 [==============================] - 0s 600us/step - loss: 24.8835 - mean_squared_error: 24.8835
Epoch 25/200
7/7 [==============================] - 0s 674us/step - loss: 24.5694 - mean_squared_error: 24.5694
Epoch 26/200
7/7 [==============================] - 0s 722us/step - loss: 24.2667 - mean_squared_error: 24.2667
Epoch 27/200
7/7 [==============================] - 0s 641us/step - loss: 23.9845 - mean_squared_error: 23.9845
Epoch 28/200
7/7 [==============================] - 0s 653us/step - loss: 23.7098 - mean_squared_error: 23.7098
Epoch 29/200
7/7 [==============================] - 0s 809us/step - loss: 23.4344 - mean_squared_error: 23.4344
Epoch

7/7 [==============================] - 0s 666us/step - loss: 19.6092 - mean_squared_error: 19.6092
Epoch 95/200
7/7 [==============================] - 0s 671us/step - loss: 19.6050 - mean_squared_error: 19.6050
Epoch 96/200
7/7 [==============================] - 0s 637us/step - loss: 19.6069 - mean_squared_error: 19.6069
Epoch 97/200
7/7 [==============================] - 0s 628us/step - loss: 19.6047 - mean_squared_error: 19.6047
Epoch 98/200
7/7 [==============================] - 0s 626us/step - loss: 19.6044 - mean_squared_error: 19.6044
Epoch 99/200
7/7 [==============================] - 0s 668us/step - loss: 19.6003 - mean_squared_error: 19.6003
Epoch 100/200
7/7 [==============================] - 0s 673us/step - loss: 19.6000 - mean_squared_error: 19.6000
Epoch 101/200
7/7 [==============================] - 0s 775us/step - loss: 19.5990 - mean_squared_error: 19.5990
Epoch 102/200
7/7 [==============================] - 0s 685us/step - loss: 19.5970 - mean_squared_error: 19.5970
Ep

...

7/7 [==============================] - 0s 663us/step - loss: 19.5879 - mean_squared_error: 19.5879
Epoch 54/200
7/7 [==============================] - 0s 644us/step - loss: 19.5845 - mean_squared_error: 19.5845
Epoch 55/200
7/7 [==============================] - 0s 616us/step - loss: 19.5917 - mean_squared_error: 19.5917
Epoch 56/200
7/7 [==============================] - 0s 623us/step - loss: 19.5882 - mean_squared_error: 19.5882
Epoch 57/200
7/7 [==============================] - 0s 616us/step - loss: 19.5857 - mean_squared_error: 19.5857
Epoch 58/200
7/7 [==============================] - 0s 628us/step - loss: 19.5839 - mean_squared_error: 19.5839
Epoch 59/200
7/7 [==============================] - 0s 624us/step - loss: 19.5843 - mean_squared_error: 19.5843
Epoch 60/200
7/7 [==============================] - 0s 670us/step - loss: 19.5797 - mean_squared_error: 19.5797
Epoch 61/200
7/7 [==============================] - 0s 590us/step - loss: 19.5872 - mean_squared_error: 19.5872
Epoch

7/7 [==============================] - 0s 614us/step - loss: 19.5844 - mean_squared_error: 19.5844
Epoch 127/200
7/7 [==============================] - 0s 696us/step - loss: 19.5853 - mean_squared_error: 19.5853
Epoch 128/200
7/7 [==============================] - 0s 676us/step - loss: 19.5814 - mean_squared_error: 19.5814
Epoch 129/200
7/7 [==============================] - 0s 604us/step - loss: 19.5840 - mean_squared_error: 19.5840
Epoch 130/200
7/7 [==============================] - 0s 638us/step - loss: 19.5871 - mean_squared_error: 19.5871
Epoch 131/200
7/7 [==============================] - 0s 599us/step - loss: 19.5867 - mean_squared_error: 19.5867
Epoch 132/200
7/7 [==============================] - 0s 681us/step - loss: 19.5852 - mean_squared_error: 19.5852
Epoch 133/200
7/7 [==============================] - 0s 570us/step - loss: 19.5805 - mean_squared_error: 19.5805
Epoch 134/200
7/7 [==============================] - 0s 629us/step - loss: 19.5863 - mean_squared_error: 19.58

7/7 [==============================] - 0s 634us/step - loss: 19.5907 - mean_squared_error: 19.5907
Epoch 199/200
7/7 [==============================] - 0s 605us/step - loss: 19.5816 - mean_squared_error: 19.5816
Epoch 200/200
7/7 [==============================] - 0s 608us/step - loss: 19.5832 - mean_squared_error: 19.5832
[0 1 3 4 6 7 8]
Epoch 1/200
13/13 [==============================] - 0s 616us/step - loss: 24.6308 - mean_squared_error: 24.6308
Epoch 2/200
13/13 [==============================] - 0s 678us/step - loss: 23.3478 - mean_squared_error: 23.3478
Epoch 3/200
13/13 [==============================] - 0s 605us/step - loss: 22.2148 - mean_squared_error: 22.2148
Epoch 4/200
13/13 [==============================] - 0s 593us/step - loss: 21.1536 - mean_squared_error: 21.1536
Epoch 5/200
13/13 [==============================] - 0s 620us/step - loss: 20.2433 - mean_squared_error: 20.2433
Epoch 6/200
13/13 [==============================] - 0s 573us/step - loss: 19.3812 - mean_squa

Epoch 70/200
13/13 [==============================] - 0s 605us/step - loss: 12.5781 - mean_squared_error: 12.5781
Epoch 71/200
13/13 [==============================] - 0s 603us/step - loss: 12.5762 - mean_squared_error: 12.5762
Epoch 72/200
13/13 [==============================] - 0s 578us/step - loss: 12.5838 - mean_squared_error: 12.5838
Epoch 73/200
13/13 [==============================] - 0s 613us/step - loss: 12.5771 - mean_squared_error: 12.5771
Epoch 74/200
13/13 [==============================] - 0s 614us/step - loss: 12.5823 - mean_squared_error: 12.5823
Epoch 75/200
13/13 [==============================] - 0s 619us/step - loss: 12.5814 - mean_squared_error: 12.5814
Epoch 76/200
13/13 [==============================] - 0s 596us/step - loss: 12.5762 - mean_squared_error: 12.5762
Epoch 77/200
13/13 [==============================] - 0s 581us/step - loss: 12.5766 - mean_squared_error: 12.5766
Epoch 78/200
13/13 [==============================] - 0s 570us/step - loss: 12.5771 - me

13/13 [==============================] - 0s 623us/step - loss: 12.5772 - mean_squared_error: 12.5772
Epoch 142/200
13/13 [==============================] - 0s 581us/step - loss: 12.5797 - mean_squared_error: 12.5797
Epoch 143/200
13/13 [==============================] - 0s 598us/step - loss: 12.5771 - mean_squared_error: 12.5771
Epoch 144/200
13/13 [==============================] - 0s 580us/step - loss: 12.5764 - mean_squared_error: 12.5764
Epoch 145/200
13/13 [==============================] - 0s 595us/step - loss: 12.5761 - mean_squared_error: 12.5761
Epoch 146/200
13/13 [==============================] - 0s 588us/step - loss: 12.5816 - mean_squared_error: 12.5816
Epoch 147/200
13/13 [==============================] - 0s 576us/step - loss: 12.5774 - mean_squared_error: 12.5774
Epoch 148/200
13/13 [==============================] - 0s 580us/step - loss: 12.5793 - mean_squared_error: 12.5793
Epoch 149/200
13/13 [==============================] - 0s 573us/step - loss: 12.5760 - mean_sq